In [1]:
import os
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import cv2
import random
import time
from PIL import Image
import pandas as pd

In [2]:
# Library Constants
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkPoints = mp.solutions.hands.HandLandmark
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
DrawingUtil = mp.solutions.drawing_utils

In [3]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)

detector = HandLandmarker.create_from_options(options)
def convert_detection_result(detection_result, image):
        """
        Draws all the landmarks on the hand
        Args:
            image (Image): Image to draw on
            detection_result (HandLandmarkerResult): HandLandmarker detection results
        """
        # Get a list of the landmarks
        hand_landmarks_list = detection_result.hand_landmarks
        
        for idx in range(len(hand_landmarks_list)):
            #NOTE this only works for one hand
            listofpointstoreturn = []
            hand_landmarks = hand_landmarks_list[idx]

            # Save the landmarks into a NormalizedLandmarkList
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
            # results = mp_hands.Hands.process(image = image)

            if hand_landmarks:
                for landmrk in hand_landmarks:
                    # for ids, landmrk in enumerate(hand_landmarks.landmark):
                        # print(ids, landmrk)
                    listofpointstoreturn.append(landmrk.x)
                    listofpointstoreturn.append(landmrk.y)
                    listofpointstoreturn.append(landmrk.z)
                return listofpointstoreturn
        
        return "Nothing Here"

I0000 00:00:1714761439.396147       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def returndatapoints(image):
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    # Draw the hand landmarks
    points = convert_detection_result(results, image)
    if points == "Nothing Here":
        border_size = 150
        frame = cv2.copyMakeBorder(image, top=border_size, 
                                bottom=border_size, left=border_size, 
                                right=border_size,
                                borderType=cv2.BORDER_CONSTANT, 
                                value=[0, 0, 0]
        )
        to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        results = detector.detect(to_detect) 
        points = convert_detection_result(results, frame)
    return points


In [5]:
def draw_landmarks_on_hand(image, detection_result):
    """
    Draws all the landmarks on the hand
    Args:
        image (Image): Image to draw on
        detection_result (HandLandmarkerResult): HandLandmarker detection results
    """
    # Get a list of the landmarks
    hand_landmarks_list = detection_result.hand_landmarks

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]

        # Save the landmarks into a NormalizedLandmarkList
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        # Draw the landmarks on the hand
        DrawingUtil.draw_landmarks(image,
                                    hand_landmarks_proto,
                                    solutions.hands.HAND_CONNECTIONS,
                                    solutions.drawing_styles.get_default_hand_landmarks_style(),
                                    solutions.drawing_styles.get_default_hand_connections_style())


In [94]:
"""
Main game loop. Runs until the 
user presses END_LETTER.
"""    
END_LETTER = "q"
# Create the hand detector
base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)
detector = HandLandmarker.create_from_options(options)

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
total_data_collector = []
CURRENT_LETTER = "y"
NUM_DATA = 20


# TODO: Load video
video = cv2.VideoCapture(0)
START_TIME = time.time()
# TODO: Modify loop condition  
while video.isOpened():
    # Get the current frame
    frame = video.read()[1]

    # Convert it to an RGB image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    # Convert the image to a readable format and find the hands
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    

    # Draw the hand landmarks
    draw_landmarks_on_hand(image, results)
    datapoints = returndatapoints(image)
    if datapoints != "Nothing Here":
        most_recent_actual_data = datapoints

    # Change the color of the frame back
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Hand Tracking", image)

    #stops data collection at NUM_DATA
    if len(total_data_collector) >= NUM_DATA:
        break
    # Break the loop if the user presses 'q'
    if cv2.waitKey(50) & 0xFF == ord(END_LETTER):
        break
    if cv2.waitKey(50) & 0xFF == ord(CURRENT_LETTER):
        print(most_recent_actual_data)
        print(len(total_data_collector))
        if datapoints != "Nothing Here":
            total_data_collector.append(most_recent_actual_data)
            
            

video.release()
cv2.destroyAllWindows()
#cv2.imshow("Hand Tracking", image)



I0000 00:00:1714762283.527265       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


[0.6402161121368408, 0.7951735854148865, -4.4987771730120585e-07, 0.5783266425132751, 0.7388153076171875, -0.019917089492082596, 0.537473201751709, 0.6343542337417603, -0.027484353631734848, 0.5057213306427002, 0.554174542427063, -0.033551547676324844, 0.4790557622909546, 0.513141930103302, -0.032404977828264236, 0.5812077522277832, 0.5174111723899841, -0.01576639525592327, 0.5796093344688416, 0.44459283351898193, -0.043231185525655746, 0.5870564579963684, 0.5257493257522583, -0.05475056916475296, 0.5876357555389404, 0.5923197269439697, -0.05649303272366524, 0.6167805194854736, 0.5158852934837341, -0.011474095284938812, 0.6154140830039978, 0.4586983919143677, -0.03891710937023163, 0.6161584258079529, 0.5681582689285278, -0.04096764326095581, 0.6131279468536377, 0.6183155179023743, -0.03502252325415611, 0.651594340801239, 0.5302727222442627, -0.01157519780099392, 0.6541312336921692, 0.4735860824584961, -0.03133593127131462, 0.6471609473228455, 0.5620114803314209, -0.020393632352352142, 

In [95]:
len(total_data_collector)

20

In [96]:
#total_data_collector[2]

In [97]:
acquired_df = []
copy_of_total_data = total_data_collector
current_letter = CURRENT_LETTER
for i in range(len(copy_of_total_data)):
    copy_of_total_data[i].append(current_letter)
    acquired_df.append(copy_of_total_data[i])



In [98]:
dfcolnames = []
for i in range(1,22):
    for j in ["x","y","z"]:
        dfcolnames.append(j+str(i))
dfcolnames.append("label")

In [99]:
copy_df = acquired_df

In [100]:
#copy_df

In [101]:
# out_list = []
# for i in range(len(copy_df)):
#     out_list.append(copy_df[i][0])

In [102]:
end_df = pd.DataFrame(copy_df, columns = dfcolnames)

In [103]:
end_df

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,x19,y19,z19,x20,y20,z20,x21,y21,z21,label
0,0.626390,0.848030,-4.448682e-07,0.577108,0.780797,-0.012959,0.556138,0.670042,-0.017565,0.550425,...,0.683310,0.553676,-0.034130,0.688919,0.505228,-0.035010,0.696868,0.462990,-0.030551,y
1,0.634696,0.842229,-4.554822e-07,0.583800,0.772402,-0.010372,0.562660,0.669885,-0.013955,0.560374,...,0.683931,0.542064,-0.036990,0.686904,0.495165,-0.039247,0.693785,0.455212,-0.034988,y
2,0.646319,0.815330,-2.603970e-07,0.576625,0.757041,-0.025438,0.533860,0.679726,-0.041512,0.504133,...,0.696950,0.489897,-0.055782,0.702292,0.437459,-0.065659,0.710575,0.392105,-0.065869,y
3,0.642818,0.804604,-2.981834e-07,0.588545,0.744425,-0.019704,0.557605,0.645009,-0.027738,0.542231,...,0.702589,0.521898,-0.048728,0.708822,0.474722,-0.054771,0.716663,0.435650,-0.052432,y
4,0.652661,0.817345,-2.852505e-07,0.594074,0.750011,-0.026081,0.556908,0.648841,-0.035698,0.535234,...,0.711859,0.546879,-0.029940,0.716804,0.502892,-0.036677,0.725657,0.465300,-0.037624,y
5,0.629281,0.794606,-1.049840e-07,0.578933,0.723034,-0.024587,0.548114,0.626843,-0.031246,0.529918,...,0.701691,0.532290,-0.010459,0.708558,0.490966,-0.012832,0.716265,0.458873,-0.010939,y
6,0.636467,0.663190,-1.984945e-07,0.606186,0.637460,-0.016105,0.595970,0.593609,-0.027341,0.596171,...,0.671561,0.522694,-0.045169,0.673633,0.484365,-0.053099,0.675256,0.451364,-0.057827,y
7,0.645863,0.781952,-3.524797e-07,0.593023,0.722308,-0.016414,0.561434,0.624280,-0.020252,0.544857,...,0.709962,0.504605,-0.031540,0.716879,0.454988,-0.039167,0.725975,0.413828,-0.040642,y
8,0.628797,0.794450,-1.787152e-07,0.574506,0.740608,-0.008253,0.541386,0.643676,-0.006869,0.524426,...,0.685267,0.536257,-0.015925,0.685135,0.490799,-0.023450,0.686999,0.455479,-0.026300,y
9,0.602660,0.690661,-2.401741e-07,0.557783,0.659772,-0.013595,0.530936,0.601513,-0.018103,0.516308,...,0.636871,0.513117,-0.019426,0.642901,0.495616,-0.022175,0.650996,0.483136,-0.020902,y


In [104]:
end_df.to_csv('data/acquired_data/' + CURRENT_LETTER + '.csv', index=False) 
              

In [126]:
# copy_df[0]